In [44]:
from ipywidgets import interact, Textarea
import nltk
from colored import fg

import kiwi
from kiwi import constants as const

In [46]:
model_file = 'en_de.torch'
model = kiwi.load_model(model_file)

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/sony/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [57]:
THRESHOLD = 0.9

GREEN = fg(10)  # GREEN
RED = fg(1)  # RED

def get_color(bad_prob):
    return GREEN if bad_prob < THRESHOLD else RED

def KiwiViz(source, mt):
    source = ' '.join(nltk.word_tokenize(source))
    mt = ' '.join(nltk.word_tokenize(mt))
    model_out = model.predict({const.SOURCE: [source.lower()], const.TARGET: [mt.lower()]})
    bad_probs = model_out[const.TARGET_TAGS][0]
    gap_probs = model_out[const.GAP_TAGS][0]
    word_colors = map(get_color, bad_probs)
    gap_colors = list(map(get_color, gap_probs[1:]))
    colored_words = [
        color + word 
        for color, word in zip(word_colors, mt.split())
    ]
    BAD_GAP = RED + '_'
    visualization = ''
    for colored_word, gap_prob in zip(colored_words, gap_probs[:-1]):
        visualization += ' ' if gap_prob < THRESHOLD else BAD_GAP + ' '
        visualization += colored_word
    visualization += '' if gap_probs[-1] < THRESHOLD else ' ' + BAD_GAP
    print('HTER: {}'.format(model_out[const.SENTENCE_SCORES][0]))
    print(visualization)

In [58]:
SOURCE = Textarea(value='stops validating after detecting 250 errors.')
MT = Textarea(value='validieren nach Entdeckung 250 Fehler stoppt.')

In [59]:
_interact = interact(KiwiViz, source=SOURCE, mt=MT)

interactive(children=(Textarea(value='stops validating after detecting 250 errors.', description='source'), Te…